In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.autograd import Variable

In [ ]:
train_features = pd.read_csv('/content/drive/My Drive/moa/train_features.csv')
train_targets_scored = pd.read_csv('/content/drive/My Drive/moa/train_targets_scored.csv')

In [ ]:
train_features.dtypes.sort_values()

X = train_features.iloc[:, 1:].values
y = train_targets_scored.iloc[:, 1:].values

In [ ]:
# oneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
X[:, 0:5]

array([[0.0, 1.0, 24, 'D1', 1.062],
       [0.0, 1.0, 72, 'D1', 0.0743],
       [0.0, 1.0, 48, 'D1', 0.628],
       ...,
       [1.0, 0.0, 48, 'D2', 0.3942],
       [0.0, 1.0, 24, 'D1', 0.666],
       [0.0, 1.0, 72, 'D1', -0.8598]], dtype=object)

In [ ]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [3])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
X[:, 0:5]

array([[1.0, 0.0, 0.0, 1.0, 24],
       [1.0, 0.0, 0.0, 1.0, 72],
       [1.0, 0.0, 0.0, 1.0, 48],
       ...,
       [0.0, 1.0, 1.0, 0.0, 48],
       [1.0, 0.0, 0.0, 1.0, 24],
       [1.0, 0.0, 0.0, 1.0, 72]], dtype=object)

In [ ]:
X.shape

(23814, 877)

In [ ]:
X

array([[1.0, 0.0, 0.0, ..., 0.2139, 0.3801, 0.4176],
       [1.0, 0.0, 0.0, ..., 0.1241, 0.6077, 0.7371],
       [1.0, 0.0, 0.0, ..., -0.2187, -1.408, 0.6931],
       ...,
       [0.0, 1.0, 1.0, ..., 0.7592, 0.6656, 0.3808],
       [1.0, 0.0, 0.0, ..., 0.7015, -0.629, 0.074],
       [1.0, 0.0, 0.0, ..., -0.4775, -2.15, -4.252]], dtype=object)

In [ ]:
# Normalization
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
X.shape

(23814, 877)

In [ ]:
y.shape

(23814, 206)

In [ ]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.5)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.5)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

# Vou inicializar no train
model = Model(877, 206, 256)
model

Model(
  (batch_norm1): BatchNorm1d(877, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (dense1): Linear(in_features=877, out_features=256, bias=True)
  (batch_norm2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (dense2): Linear(in_features=256, out_features=256, bias=True)
  (batch_norm3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout3): Dropout(p=0.5, inplace=False)
  (dense3): Linear(in_features=256, out_features=206, bias=True)
)

In [ ]:
# train_loader = torch.utils.data.DataLoader(
#     train_dataset, batch_size=1024, num_workers=8
# )

def train(self, data_loader):
    self.model.train()
    final_loss = 0
    for data in data_loader:
        self.optimizer.zero_grad()
        inputs = data["x"].to(self.device)
        targets = data["x"].to(self.device)
        outputs = self.model(inputs)
        loss = self.loss_fn(targets, outputs)
        loss.backward()
        self.optimizer.step()
        final_loss += loss.item()
    return final_loss / len(data_loader)

In [ ]:
class Engine:
    def __init__(self, model, optimizer, device):
        self.model = model
        self.optimizer = optimizer
        self.device = device

    @staticmethod
    def loss_fn(targets, outputs):
        return nn.BCEWithLogitsLoss()(outputs, targets)

    def train(self, data_loader):
        self.model.train()
        final_loss = 0
        for data in data_loader:
            self.optimizer.zero_grad()
            inputs = data["x"].to(self.device)
            targets = data["x"].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(targets, outputs)
            loss.backward()
            self.optimizer.step()
            final_loss += loss.item()
        return final_loss / len(data_loader)


# Train
DEVICE = "cuda"
EPOCHS = 100

def run_training():
    # df = pd.read_csv("train_features.csv")
    # df = utils.process_data(df)
    # folds = pd.read_csv("train_folds.csv")

    # targets = folds.drop(["sig_id", "kfold"], axis=1).columns
    # features = df.drop("sig_id", axis=1).columns

    # df = df.merge(folds, on="sig_id", how="left")

    # train_df = df[df.kfold != fold].reset_index(drop=True) 
    # valid_df = df[df.kfold == fold].reset_index(drop=True) 

    # x_train = train_df[features].to_array()
    # x_valid = valid_df[features].to_array()

    # y_train = train_df[targets].to_array()
    # y_valid = valid_df[targets].to_array()

    # train_dataset = utils.MoaDataset(x_train, y_train)
    # train_loader = torch.utils.data.DataLoader(
    #     train_dataset, batch_size=1024, num_workers=8
    # )

    model = Model(877, 206)
    model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
    # scheduler = 

    eng = Engine(
        model, optimizer, DEVICE
    )

    for _ in range(EPOCHS):
        train_loss = eng.train(train_loss)
        # valid_loss = ......

run_training()

UnboundLocalError: ignored

In [ ]:
# loss_fn = nn.BCEWithLogitsLoss()                       
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# batch_size = 64
# n_epochs = 100
# batch_no = len(X) // batch_size                 # batch_no=125, pois 125 * 64 = 8000

# train_loss = 0
# train_loss_min = np.Inf
# for epoch in range(n_epochs):
#     for i in range(batch_no):
#         start = i * batch_size
#         end = start + batch_size
#         x_var = Variable(torch.FloatTensor(X[start:end]))
#         y_var = Variable(torch.FloatTensor(y[start:end])) 
        
        
#         # output = model(x_var)
#         # loss = loss_fn(output,y_var)

#         optimizer.zero_grad()
#         output = model(x_var)       
#         loss = loss_fn(output,y_var)
#         loss.backward()
#         optimizer.step()
        
#         # values, labels = torch.max(output, 1)
#         # num_right = np.sum(labels.data.numpy() == y[start:end])
#         train_loss += loss.item() * batch_size
    
#     train_loss = train_loss / len(X)
#     if train_loss <= train_loss_min:
#         print("Validation loss decreased ({:6f} ===> {:6f}). Saving the model...".format(train_loss_min,train_loss))
#         torch.save(model.state_dict(), "model.pt")
#         train_loss_min = train_loss
    
#     # if epoch % 200 == 0:
#     #     print('')
#     #     print("Epoch: {} \tTrain Loss: {} \tTrain Accuracy: {}".format(epoch + 1, train_loss))
#     print("Ronaldo")

# print('Training Ended! ')

